# Caso Práctico: Sistema Inteligente de Clasificación de Reseñas para "GastroReseñas"

**Empresa  GastroReseñas Contexto**: "GastroReseñas" es una plataforma web líder en la recopilación de opiniones sobre restaurantes. Diariamente, miles de usuarios publican reseñas sobre sus experiencias gastronómicas. Actualmente, la plataforma solo muestra una calificación de 1 a 5 estrellas.

La empresa quiere lanzar una nueva funcionalidad: un "resumen de opinión" que etiquete automáticamente cada reseña con aspectos clave mencionados por el usuario. Esto permitiría a los nuevos clientes filtrar, por ejemplo, por reseñas que hablen positivamente del "Servicio" o negativamente del "Precio".




**El Desafío**: El volumen de reseñas es demasiado grande para ser etiquetado manualmente. El equipo de "GastroReseñas" ha recopilado un conjunto de datos (un archivo .csv) donde han etiquetado manualmente 10,000 reseñas de ejemplo. El problema es que una misma reseña puede referirse a múltiples aspectos. Por ejemplo:

- Reseña: "La comida estaba deliciosa, pero el camarero fue muy lento."

- Etiquetas deseadas: [Comida_Positivo], [Servicio_Negativo]

Esto se conoce como un problema de clasificación de texto multietiqueta (multi-label classification).

## Enunciado del Problema y Tarea 

Tu objetivo es desarrollar un sistema basado en un modelo Transformer (como BERT, DistilBERT o alguno similar que hayamos dado en clase) capaz de leer el texto de una nueva reseña de restaurante y asignar todas las etiquetas que correspondan de una lista predefinida.


Conjunto de Datos: Se te proporciona un archivo gastro_reseñas_etiquetadas.csv. Las columnas relevantes son:

- id_reseña: Identificador único.

- texto_reseña: El texto completo de la opinión del usuario.

- etiqueta_comida_pos: (1 o 0)

- etiqueta_comida_neg: (1 o 0)

- etiqueta_servicio_pos: (1 o 0)

- etiqueta_servicio_neg: (1 o 0)

- etiqueta_ambiente_pos: (1 o 0)

- etiqueta_ambiente_neg: (1 o 0)

- etiqueta_precio_pos: (1 o 0)

- etiqueta_precio_neg: (1 o 0)

(Nota: Un 1 indica que la etiqueta aplica, un 0 que no aplica. Una misma fila puede tener múltiples 1s).

## Requisitos del Proyecto



Debes configurar un script o notebook de Python que implemente la solución completa utilizando la biblioteca transformers (de Hugging Face) y un framework como PyTorch o TensorFlow.



## Pasos Obligatorios


1. Carga y Preparación de Datos:

- Cargar el archivo gastro_reseñas_etiquetadas.csv (usando pandas).

- Preparar los datos para el modelo. Dado que es un problema multietiqueta, debes consolidar las columnas de etiquetas en un único formato que el modelo pueda entender (p.ej., un array de 8 dimensiones por cada reseña, como [1, 0, 0, 1, 0, 0, 0, 0]).

- Dividir los datos en conjuntos de entrenamiento, validación y prueba.

2. Tokenización:

- Elegir e instanciar un tokenizador pre-entrenado (ej. DistilBertTokenizerFast o cualquier otro que hayais utilizado en clase).

- Tokenizar los texto_reseña para que el modelo Transformer pueda procesarlos (generando input_ids y attention_mask).

3. Configuración del Modelo:

- Cargar un modelo Transformer pre-entrenado (ej. DistilBertForSequenceClassification o cualquier otro que hayais utilizado en clase) desde la biblioteca transformers.

- Importante: Al cargar el modelo, deberás configurarlo específicamente para un problema de clasificación multietiqueta (multi-label), asegurándote de que la capa de salida tenga 8 neuronas (una por cada etiqueta posible) y utilice una función de activación y pérdida adecuadas (p.ej., Sigmoide en la salida y BCEWithLogitsLoss como función de pérdida).

4. Entrenamiento (Fine-Tuning):

- Configurar los TrainingArguments (o un bucle de entrenamiento manual).

- Implementar el "fine-tuning" del modelo sobre el conjunto de datos de entrenamiento, monitorizando la pérdida en el conjunto de validación.

5. Evaluación:

- Una vez entrenado el modelo, usar el conjunto de prueba (datos que el modelo nunca ha visto) para evaluar su rendimiento.

- Calcular métricas apropiadas para clasificación multietiqueta, como el F1-Score (micro y macro), Precisión (micro) y Recall (micro).

Nota: No se puede usar 'accuracy' simple, ya que una predicción puede ser parcialmente correcta (acertar 3 de 8 etiquetas).

6. Función de Inferencia:

- Crear una función final llamada clasificar_reseña(texto) que reciba un string con una nueva reseña y devuelva una lista con los nombres de las etiquetas predichas (ej. ['Comida_Positivo', 'Servicio_Negativo']).